In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import 
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns

In [17]:
Train_data = pd.read_csv('train.csv')
Test_data = pd.read_csv('test.csv')
train_data = Train_data.copy()
test_data = Test_data.copy()

In [18]:
def preproc(df):
    df['date'] = pd.to_datetime(train_data['date'])
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['dow'] = df['date'].dt.dayofweek
    df = pd.get_dummies(df,columns=['country','product','store'])
    return df
train_data = preproc(train_data)
test_data = preproc(test_data)

In [5]:
data = train_data.drop(columns = ['num_sold','row_id','date'])
target = train_data['num_sold']

from sklearn.preprocessing import StandardScaler
Normalize = StandardScaler()
data = Normalize.fit_transform(data)
target = np.log(target)

In [6]:
train_data.head(10)


,row_id,date,num_sold,year,month,day,dow,country_Finland,country_Norway,country_Sweden,product_Kaggle Hat,product_Kaggle Mug,product_Kaggle Sticker,store_KaggleMart,store_KaggleRama
0,0,2015-01-01,329,2015,1,1,3,1,0,0,0,1,0,1,0
1,1,2015-01-01,520,2015,1,1,3,1,0,0,1,0,0,1,0
2,2,2015-01-01,146,2015,1,1,3,1,0,0,0,0,1,1,0
3,3,2015-01-01,572,2015,1,1,3,1,0,0,0,1,0,0,1
4,4,2015-01-01,911,2015,1,1,3,1,0,0,1,0,0,0,1
5,5,2015-01-01,283,2015,1,1,3,1,0,0,0,0,1,0,1
6,6,2015-01-01,526,2015,1,1,3,0,1,0,0,1,0,1,0
7,7,2015-01-01,906,2015,1,1,3,0,1,0,1,0,0,1,0
8,8,2015-01-01,250,2015,1,1,3,0,1,0,0,0,1,1,0
9,9,2015-01-01,1005,2015,1,1,3,0,1,0,0,1,0,0,1


In [11]:
import pycaret.regression
amldf = train_data.drop(columns = ['row_id','date'])



In [12]:
exp_reg = pycaret.regression.setup(amldf, target = 'num_sold')

,Description,Value
0,session_id,4916
1,Target,num_sold
2,Original Data,"(26298, 13)"
3,Missing Values,False
4,Numeric Features,9
5,Categorical Features,3
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(18408, 31)"


In [14]:
pycaret.regression.compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,18.1486,861.6156,29.2818,0.9876,0.0587,0.0465,6.1700
xgboost,Extreme Gradient Boosting,19.9135,1095.1961,32.9372,0.9843,0.0656,0.0514,0.6270
lightgbm,Light Gradient Boosting Machine,22.0490,1426.2857,37.5478,0.9795,0.0709,0.0555,33.0220
rf,Random Forest Regressor,26.0734,2509.1390,49.5736,0.9641,0.0870,0.0625,0.7230
et,Extra Trees Regressor,29.3771,3384.6301,57.5104,0.9514,0.1000,0.0704,0.8020
dt,Decision Tree Regressor,34.0181,4420.7464,66.0448,0.9363,0.1137,0.0812,0.0360
gbr,Gradient Boosting Regressor,36.9408,4662.7309,67.9719,0.9330,0.1209,0.0921,0.2390
lr,Linear Regression,79.7060,13707.8119,116.8811,0.8023,0.5076,0.2840,0.6590
ridge,Ridge Regression,79.6942,13707.8142,116.8810,0.8023,0.5077,0.2839,0.0750
br,Bayesian Ridge,79.6848,13707.8004,116.8809,0.8023,0.5077,0.2838,0.1160


In [21]:
cbm =  pycaret.regression.create_model('catboost')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,18.6727,961.1631,31.0026,0.9862,0.0606,0.0476
1,17.7967,856.2871,29.2624,0.9868,0.0580,0.0459
2,18.3219,877.8490,29.6285,0.9873,0.0608,0.0479
3,17.8181,790.9795,28.1244,0.9875,0.0589,0.0465
4,17.3245,712.6325,26.6952,0.9886,0.0582,0.0459
5,19.0702,1161.1680,34.0759,0.9853,0.0582,0.0463
6,18.5582,823.7047,28.7003,0.9884,0.0600,0.0478
7,18.2482,880.2499,29.6690,0.9882,0.0578,0.0454
8,17.2662,706.5154,26.5804,0.9897,0.0564,0.0450
9,18.4093,845.6069,29.0793,0.9878,0.0586,0.0467


In [27]:
prdecit_model = pycaret.regression.predict_model(cbm, data = test_data.drop(columns = ['row_id','date']))
#test_data.head()

In [42]:
prdecit_model.join(test_data['row_id'])

,year,month,day,dow,country_Finland,country_Norway,country_Sweden,product_Kaggle Hat,product_Kaggle Mug,product_Kaggle Sticker,store_KaggleMart,store_KaggleRama,Label,row_id
0,2015,1,1,3,1,0,0,0,1,0,1,0,363.793170,26298
1,2015,1,1,3,1,0,0,1,0,0,1,0,538.356291,26299
2,2015,1,1,3,1,0,0,0,0,1,1,0,135.468215,26300
3,2015,1,1,3,1,0,0,0,1,0,0,1,628.722116,26301
4,2015,1,1,3,1,0,0,1,0,0,0,1,926.031054,26302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6565,2015,12,31,3,0,0,1,1,0,0,1,0,754.597360,32863
6566,2015,12,31,3,0,0,1,0,0,1,1,0,206.598530,32864
6567,2015,12,31,3,0,0,1,0,1,0,0,1,837.504235,32865
6568,2015,12,31,3,0,0,1,1,0,0,0,1,1308.893451,32866


In [44]:
submission = pd.read_csv('sample_submission.csv')
submission['num_sold'] = prdecit_model['Label']
submission.to_csv('submission.csv', index=False)